In [1]:
import torch
import gluonnlp as nlp     # GluonNLP는 버트를 간단하게 로딩하는 인터페이스를 제공하는 API 임
import numpy as np
from transformers import BertTokenizer, BertTokenizerFast, BertModel

import sys
sys.path.append("..")
from myutils import seed_everything, GPU_info, pytorch_cos_sim

logfilepath:../../log/bwdataset_2022-04-15.log
logfilepath:../../log/qnadataset_2022-04-15.log


In [2]:
'''
#config.json, pytorch_model.bin 같이 있는 폴더 지정
model_path = 'model/bert-multilingual-cased_furter_pt_model_0216' 
# voab.txt, special_tokens_map.json, tokenizer_config.json,added_tokens.json 폴더 경로 지정
vocab_path = "model/bert-multilingual-cased_furter_pt_model_0216/vocab"
'''

model_path = "../../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327"
vocab_path = "../../model/bert/bmc-fpt-wiki_20190620_mecab_false_0311-nouns-0327"

# True로 해야, hidden_states 가 출력됨
output_hidden_states = True
return_dict = False

seed = 111

In [ ]:
cuda = GPU_info()
print(cuda)

In [ ]:
#seed 설정
seed_everything(seed)

In [ ]:
# model 불러옴
model = BertModel.from_pretrained(model_path, 
                                  output_hidden_states=output_hidden_states,
                                  return_dict=return_dict)

In [ ]:
device = torch.device(cuda)
#model.to(device)
model.eval()

In [ ]:
model.num_parameters()

In [3]:
# tokenize 설정
tokenizer = BertTokenizerFast.from_pretrained(vocab_path)

In [ ]:
len(tokenizer)

In [ ]:
#list(tokenizer.vocab.keys())[8000:8010]

In [17]:
#tokenized_input = tokenizer("식당에 가서 밥을 배 부르게 먹고 낙시배를 타고 고기 잡고 요트배를 타고 관광을 해야 겠다", return_tensors="pt")
#text = "식민지 관리들은 1680년대로 거슬러 올라가, 레나페 델라웨어 족이 델라웨어 강과 리하이 강 합류 (현재 라이츠 타운 근처)하는 지점에서 시작하여 한 남자가 하루 반을 도보할 수 있는 곳까지’ 토지를 매각하겠다고 약속했다고 주장했다. "
text = '''
미국 의 관타나모 수용소 문제 , 세계 의 인권 문제 에서 도 관심 이 깊 어 유엔 에 유엔 인권 고등 판무관 의 제도 를 시행 하 도록 노력 하 여 독재자 들 의 인권 유린 에 대해 제약 을 하 고 , 
국제 형사 재판소 를 만드 는 데 기여 하 여 독재자 들 같 은 인권 유린 범죄자 를 재판소 로 회부 하 여 국제 적 인 처벌 을 받 게 하 는 등 인권 신장 에 크 나 큰 기여 를 했 다 .
'''
tokenized_input = tokenizer(text, padding="max_length", max_length=96, return_tensors="pt")

In [18]:

token_str = [[tokenizer.convert_ids_to_tokens(s) for s in tokenized_input['input_ids'].tolist()[0]]]

# token indexs ( 토큰을 index로 변한한 값)
token_ids = [tokenized_input['input_ids'].tolist()[0]]
# attention_mask (중요토큰 : 1)
token_attention_mask = [tokenized_input['attention_mask'].tolist()[0]]
# segment_id (첫번째문자:0, 다음 문장:1)
token_type_ids = [tokenized_input['token_type_ids'].tolist()[0]]

print(len(token_type_ids[0]))
print(token_str)
print(token_ids)
print(token_attention_mask)
print(token_type_ids)


96
[['[CLS]', '미국', '의', '관', '##타', '##나', '##모', '수용소', '문제', ',', '세계', '의', '인권', '문제', '에서', '도', '관심', '이', '깊', '어', '유엔', '에', '유엔', '인권', '고등', '판', '##무관', '의', '제도', '를', '시행', '하', '도', '##록', '노력', '하', '여', '독재자', '들', '의', '인권', '유린', '에', '대해', '제약', '을', '하', '고', ',', '국제', '형사', '재판소', '를', '만', '##드', '는', '데', '기여', '하', '여', '독재자', '들', '같', '은', '인권', '유린', '범죄자', '를', '재판소', '로', '회부', '하', '여', '국제', '적', '인', '처벌', '을', '받', '게', '하', '는', '등', '인권', '신장', '에', '크', '나', '큰', '기여', '를', '했', '다', '.', '[SEP]', '[PAD]']]
[[101, 23545, 9637, 8900, 22695, 16439, 39420, 123574, 119564, 117, 32613, 9637, 120922, 119564, 24178, 9087, 119798, 9638, 8938, 9546, 120764, 9559, 120764, 120922, 121662, 9903, 135408, 9637, 119737, 9233, 120058, 9952, 9087, 31398, 119840, 9952, 9565, 127954, 9117, 9637, 120922, 131955, 9559, 33378, 122304, 9633, 9952, 8888, 117, 61320, 121528, 128564, 9233, 9248, 15001, 9043, 9083, 120127, 9952, 9565, 127954, 9117, 8855, 9632, 120922, 13195

In [8]:
len(token_attention_mask[0])

263

In [ ]:
tokenizer.convert_ids_to_tokens(6313)

In [ ]:
print(tokenized_input)

In [ ]:
outputs = model(**tokenized_input)

In [ ]:
print(len(outputs))

In [ ]:
sequence_output = outputs[0]
print('sequence 길이: {}'.format(sequence_output.size()))

pooled_output = outputs[1]
print('pooled 길이:{}'.format(pooled_output.size()))

hidden_states = outputs[2]
layer_idx = 0
batch_idx = 0
token_idx = 0
print('hidden_states')
print("-레이어 수:{}".format(len(hidden_states)))
print("-배치 수: {}".format(len(hidden_states[layer_idx])))
print("-토큰 수 : {}".format(len(hidden_states[layer_idx][batch_idx])))
print("-hidden 유닛 수 : {}".format(len(hidden_states[layer_idx][batch_idx][token_idx])))

In [ ]:
##### 단어별 유사도 측정

# premute를 사용하여 레이어 와 tokens 차원을 바꾼다.
sequence_output_embedding = sequence_output.permute(1,0,2)
print('sequence_output size: {}'.format(sequence_output_embedding.size()))

# tensor -> list 로 변환
output_list = sequence_output_embedding.tolist()

word_1 = 8
word_2 = 15
word_3 = 25
simul_score1 = pytorch_cos_sim(output_list[word_1][0], output_list[word_2][0])
simul_score2 = pytorch_cos_sim(output_list[word_1][0], output_list[word_3][0])
simul_score3 = pytorch_cos_sim(output_list[word_2][0], output_list[word_3][0])

print("몸에 있는배 vs 낙시배 유사도:{}".format(simul_score1))
print("몸에 있는배 vs 요트배 유사도:{}".format(simul_score2))
print("낙시배 vs 요트배 유사도:{}".format(simul_score3))

In [ ]:
print(output_list[6][0])